# Trabajo Práctico Final: Linear/Quadratic Discriminant Analysis (LDA/QDA)

### Definición: Clasificador Bayesiano

Sean $k$ poblaciones, $x \in \mathbb{R}^p$ puede pertenecer a cualquiera $g \in \mathcal{G}$ de ellas. Bajo un esquema bayesiano, se define entonces $\pi_j \doteq P(G = j)$ la probabilidad *a priori* de que $X$ pertenezca a la clase *j*, y se **asume conocida** la distribución condicional de cada observable dado su clase $f_j \doteq f_{X|G=j}$.

De esta manera dicha probabilidad *a posteriori* resulta
$$
P(G = j|X=x) = \frac{P(X=x|G = j) P(G = j)}{P(X=x)} = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

$$
P(G|_{X=x} = j) = \frac{f_{X|G=j}(x) \cdot p_G(j)}{f_X(x)} \propto f_j(x) \cdot \pi_j
$$

La regla de decisión de Bayes es entonces
$$
H(x) \doteq \arg \max_{g \in \mathcal{G}} \{ P(G = g | X=x) \} = \arg \max_{g \in \mathcal{G}} \{ f_g(x) \cdot \pi_g \}
$$

 

es decir, se predice a $x$ como perteneciente a la población $j$ cuya probabilidad a posteriori es máxima.

*Ojo, a no desesperar! $\pi_j$ no es otra cosa que una constante prefijada, y $f_j$ es, en su esencia, un campo escalar de $x$ a simplemente evaluar.*

### Distribución condicional

Para los clasificadores de discriminante cuadrático y lineal (QDA/LDA) se asume que $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma_j)$, es decir, se asume que cada población sigue una distribución normal.

Por definición, se tiene entonces que para una clase $j$:
$$
f_j(x) = \frac{1}{(2 \pi)^\frac{p}{2} \cdot |\Sigma_j|^\frac{1}{2}} e^{- \frac{1}{2}(x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j)}
$$

Aplicando logaritmo (que al ser una función estrictamente creciente no afecta el cálculo de máximos/mínimos), queda algo mucho más práctico de trabajar:

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

Observar que en este caso $C=-\frac{p}{2} \log(2\pi)$, pero no se tiene en cuenta ya que al tener una constante aditiva en todas las clases, no afecta al cálculo del máximo.

### LDA

En el caso de LDA se hace una suposición extra, que es $X|_{G=j} \sim \mathcal{N}_p(\mu_j, \Sigma)$, es decir que las poblaciones no sólo siguen una distribución normal sino que son de igual matriz de covarianzas. Reemplazando arriba se obtiene entonces:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$



Ahora, como $-\frac{1}{2}\log |\Sigma|$ es común a todas las clases se puede incorporar a la constante aditiva y, distribuyendo y reagrupando términos sobre $(x-\mu_j)^T \Sigma^{-1} (x- \mu_j)$ se obtiene finalmente:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

### Entrenamiento/Ajuste

Obsérvese que para ambos modelos, ajustarlos a los datos implica estimar los parámetros $(\mu_j, \Sigma_j) \; \forall j = 1, \dots, k$ en el caso de QDA, y $(\mu_j, \Sigma)$ para LDA.

Estos parámetros se estiman por máxima verosimilitud, de manera que los estimadores resultan:

* $\hat{\mu}_j = \bar{x}_j$ el promedio de los $x$ de la clase *j*
* $\hat{\Sigma}_j = s^2_j$ la matriz de covarianzas estimada para cada clase *j*
* $\hat{\pi}_j = f_{R_j} = \frac{n_j}{n}$ la frecuencia relativa de la clase *j* en la muestra
* $\hat{\Sigma} = \frac{1}{n} \sum_{j=1}^k n_j \cdot s^2_j$ el promedio ponderado (por frecs. relativas) de las matrices de covarianzas de todas las clases. *Observar que se utiliza el estimador de MV y no el insesgado*

Es importante notar que si bien todos los $\mu, \Sigma$ deben ser estimados, la distribución *a priori* puede no inferirse de los datos sino asumirse previamente, utilizándose como entrada del modelo.

### Predicción

Para estos modelos, al igual que para cualquier clasificador Bayesiano del tipo antes visto, la estimación de la clase es por método *plug-in* sobre la regla de decisión $H(x)$, es decir devolver la clase que maximiza $\hat{f}_j(x) \cdot \hat{\pi}_j$, o lo que es lo mismo $\log\hat{f}_j(x) + \log\hat{\pi}_j$.

## Estructura del código

## Modelo 

In [2]:
import numpy as np
from numpy.linalg import det, inv

In [3]:
class ClassEncoder:
  def fit(self, y):
    self.names = np.unique(y)
    self.name_to_class = {name:idx for idx, name in enumerate(self.names)}
    self.fmt = y.dtype
    # Q1: why is there no need for class_to_name?
    # el proceso de decodificación se realiza mediante 
    # la matriz self.names que se indexa utilizando los índices de clase directamente.

  def _map_reshape(self, f, arr):
    return np.array([f(elem) for elem in arr.flatten()]).reshape(arr.shape)
    # Q2: why is reshaping necessary?
    # esto garantiza que la matriz de salida tenga la misma forma que la matriz de entrada.

  def transform(self, y):
    return self._map_reshape(lambda name: self.name_to_class[name], y)

  def fit_transform(self, y):
    self.fit(y)
    return self.transform(y)

  def detransform(self, y_hat):
    return self._map_reshape(lambda idx: self.names[idx], y_hat)

In [4]:
class BaseBayesianClassifier:
  def __init__(self):
    self.encoder = ClassEncoder()

  def _estimate_a_priori(self, y):
    a_priori = np.bincount(y.flatten().astype(int)) / y.size
    # Q3: what does bincount do?
    return np.log(a_priori)
    # Segun la documentación de numpy: cuenta las ocurrencias de números enteros no negativos en un array.
    # O sea se utiliza para contar las ocurrencias de cada clase en y.
    
  def _fit_params(self, X, y):
    # estimate all needed parameters for given model
    raise NotImplementedError()

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    raise NotImplementedError()

  def fit(self, X, y, a_priori=None):
    # first encode the classes
    y = self.encoder.fit_transform(y)

    # if it's needed, estimate a priori probabilities
    self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)

    # check that a_priori has the correct number of classes
    assert len(self.log_a_priori) == len(self.encoder.names), "A priori probabilities do not match number of classes"

    # now that everything else is in place, estimate all needed parameters for given model
    self._fit_params(X, y)
    # Q4: why do we need to do this last, can't we do it first?
    # No, porque antes de de estimar los parámetros del modelo, hay que
    # codificar las clases y calcular las probabilidades a priori.

  def predict(self, X):
    # this is actually an individual prediction encased in a for-loop
    m_obs = X.shape[1]
    y_hat = np.empty(m_obs, dtype=self.encoder.fmt)
    
    for i in range(m_obs):
      encoded_y_hat_i = self._predict_one(X[:,i].reshape(-1,1))
      y_hat[i] = self.encoder.names[encoded_y_hat_i]

    # return prediction as a row vector (matching y)
    return y_hat.reshape(1,-1)

  def _predict_one(self, x):
    # calculate all log posteriori probabilities (actually, +C)
    log_posteriori = [ log_a_priori_i + self._predict_log_conditional(x, idx) for idx, log_a_priori_i 
                  in enumerate(self.log_a_priori) ]

    # return the class that has maximum a posteriori probability
    return np.argmax(log_posteriori)

In [5]:
class QDA(BaseBayesianClassifier):

  def _fit_params(self, X, y):
    # estimate each covariance matrix
    self.inv_covs = [inv(np.cov(X[:,y.flatten()==idx], bias=True)) 
                      for idx in range(len(self.log_a_priori))]
    # Q5: why not just X[:,y==idx]?
    # flatten() aplana la matriz 'y' y la convierte en un array unidimensional.
    # en numpy cuando se trabaja con matrices de diferentes dimensiones, a veces
    # puede haber problemas con el broadcasting, o sea la forma en la que numpy trata 
    # de combinar matrices de diferentes dimensiones para realizar operaciones. 
    # Convirtiendo la matriz 'y' en un array unidimensional se simplifican las operaciones
    # ya que se asegura que ambas matrices de la comparación tengan dimensiones compatibles.

    
    # Q6: what does bias=True mean? why not use bias=False?
    # significa que se utiliza la formula de covarianza con sesgo


    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True) 
                  for idx in range(len(self.log_a_priori))]
    # Q7: what does axis=1 mean? why not axis=0 instead?

    # esto significa que se calcula la media a lo largo del eje x, o sea las columnas
    # para cada feature. Por lo que resultaen un vector columna con la media de cada feature.
    # No se utiliza axis = 0 porque eso implicaría usar la media a lo largo del eje vertical, 
    # es decir las filas, y lo que se quiere es tener la media de cada feature por separado.

  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    inv_cov = self.inv_covs[class_idx]
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x

In [6]:
class LDA(BaseBayesianClassifier):
    def _fit_params(self, X, y):
        self.means = [X[:, y.flatten() == idx].mean(axis=1, keepdims=True)
                      for idx in range(len(self.log_a_priori))]
        
        cov = sum([np.cov(X[:, y.flatten() == idx], bias=True)
                  for idx in range(len(self.log_a_priori))])
        self.inv_cov = inv(cov)

    def _predict_log_conditional(self, x, class_idx):
        unbiased_x = x - self.means[class_idx]
        return -0.5 * unbiased_x.T @ self.inv_cov @ unbiased_x

    def _predict_one(self, x):
        log_posteriori = [log_a_priori_i + self._predict_log_conditional(x, idx).item()
                          for idx, log_a_priori_i in enumerate(self.log_a_priori)]
        return np.argmax(log_posteriori)

## Código para pruebas

In [7]:
# hiperparámetros
#rng_seed = 6543
#rng_seed = 1000
rng_seed = 120

In [8]:
from sklearn.datasets import load_iris

def get_iris_dataset():
  data = load_iris()
  X_full = data.data
  y_full = np.array([data.target_names[y] for y in data.target.reshape(-1,1)])
  return X_full, y_full

X_full, y_full = get_iris_dataset()

print(f"X: {X_full.shape}, Y:{y_full.shape}")

X: (150, 4), Y:(150, 1)


In [9]:
# peek data matrix
X_full[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [10]:
# peek target vector
y_full[:5]

array([['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa'],
       ['setosa']], dtype='<U10')

In [11]:
# preparing data, train - test validation
# 70-30 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.4, random_state=rng_seed)

# traspose everything because this model format assumes column vectors
train_x = X_train.T
train_y = y_train.T
test_x = X_test.T
test_y = y_test.T

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(4, 90) (1, 90) (4, 60) (1, 60)


In [12]:
#Análisis de datos
unique_classes, counts = np.unique(train_y, return_counts=True)

# Imprimir la cantidad de muestras para cada clase
for unique_class, count in zip(unique_classes, counts):
    print(f"Clase: {unique_class}, Cantidad de muestras: {count}")

# Calcular y mostrar la proporción entre clases
proportions = counts / counts.sum()
print(f"Proporciones de clases: {proportions}")

Clase: setosa, Cantidad de muestras: 26
Clase: versicolor, Cantidad de muestras: 31
Clase: virginica, Cantidad de muestras: 33
Proporciones de clases: [0.28888889 0.34444444 0.36666667]


### QDA

In [13]:
#Entrenar el modelo con una distribución a priori normal
qda_normal = QDA()
qda_normal.fit(train_x, train_y)

In [14]:
# Realizar predicciones con la distribución a priori normal
train_y_pred_normal = qda_normal.predict(train_x)
test_y_pred_normal = qda_normal.predict(test_x)

In [15]:
def accuracy(y_true, y_pred):
  return (y_true == y_pred).mean()

In [16]:
# Calcular la accuracy del modelo con la distribución a priori normal
train_acc_normal = accuracy(train_y, train_y_pred_normal)
test_acc_normal = accuracy(test_y, test_y_pred_normal)

In [17]:
#Entrenar el modelo con una distribución a priori uniforme
qda_uniforme = QDA()

# Calcular las probabilidades a priori uniformes
n_classes = len(np.unique(y_full))
a_priori_uniform = np.full(n_classes, 1 / n_classes)

qda_uniforme.fit(train_x, train_y, a_priori=a_priori_uniform)

In [18]:
# Realizar predicciones con la distribución a priori uniforme
train_y_pred_uniforme = qda_uniforme.predict(train_x)
test_y_pred_uniforme = qda_uniforme.predict(test_x)

In [19]:
# Calcular la accuracy del modelo con la distribución a priori uniforme
train_acc_uniforme = accuracy(train_y, train_y_pred_uniforme)
test_acc_uniforme = accuracy(test_y, test_y_pred_uniforme)

In [20]:
#Comparar accuracy de QDA con distribución a priori normal vs distribución uniforme
print(f"QDA (normal a priori): Train (apparent) error is {1-train_acc_normal:.4f} while test error is {1-test_acc_normal:.4f}")
print(f"QDA (uniforme a priori): Train (apparent) error is {1-train_acc_uniforme:.4f} while test error is {1-test_acc_uniforme:.4f}")

QDA (normal a priori): Train (apparent) error is 0.0111 while test error is 0.0167
QDA (uniforme a priori): Train (apparent) error is 0.0111 while test error is 0.0167


### LDA

In [ ]:
#LDA
lda_normal = LDA()
lda_normal.fit(train_x, train_y)

In [ ]:
# Realizar predicciones con la distribución a priori normal
train_y_pred_lda_normal = lda_normal.predict(train_x)
test_y_pred_lda_normal = lda_normal.predict(test_x)

In [ ]:
# Calcular la accuracy del modelo con la distribución a priori normal
train_acc_lda_normal = accuracy(train_y, train_y_pred_lda_normal)
test_acc_lda_normal = accuracy(test_y, test_y_pred_lda_normal)

In [ ]:
#Entrenar el modelo con una distribución a priori uniforme
lda_uniforme = LDA()
lda_uniforme.fit(train_x, train_y, a_priori=a_priori_uniform)

In [ ]:
# Realizar predicciones con la distribución a priori uniforme
train_y_pred_lda_uniforme = lda_uniforme.predict(train_x)
test_y_pred_lda_uniforme = lda_uniforme.predict(test_x)

In [ ]:
# Calcular la accuracy del modelo con la distribución a priori uniforme
train_acc_lda_uniforme = accuracy(train_y, train_y_pred_lda_uniforme)
test_acc_lda_uniforme = accuracy(test_y, test_y_pred_lda_uniforme)

In [ ]:
#Comparar accuracy de LDA con distribución a priori normal vs distribución uniforme
print(f"LDA (normal a priori): Train (apparent) error is {1-train_acc_lda_normal:.4f} while test error is {1-test_acc_lda_normal:.4f}")
print(f"LDA (uniforme a priori): Train (apparent) error is {1-train_acc_lda_uniforme:.4f} while test error is {1-test_acc_lda_uniforme:.4f}")

## Resultados

#### ***Random seed=6543***

* QDA (normal a priori): Train (apparent) error is 0.0111 while test error is 0.0167

* QDA Uniforme: Train (apparent) error is 0.0222 while test error is 0.0167

* LDA (normal a priori): Train (apparent) error is 0.0111 while test error is 0.0167

* LDA (uniforme a priori): Train (apparent) error is 0.0222 while test error is 0.0167

#### ***Random seed=1000***

* QDA (normal a priori): Train (apparent) error is 0.0111 while test error is 0.0333

* QDA (uniforme a priori): Train (apparent) error is 0.0111 while test error is 0.0333

* LDA (normal a priori): Train (apparent) error is 0.0111 while test error is 0.0500

* LDA (uniforme a priori): Train (apparent) error is 0.0111 while test error is 0.0500 


#### ***Random seed=120***

* QDA (normal a priori): Train (apparent) error is 0.0111 while test error is 0.0167

* QDA (uniforme a priori): Train (apparent) error is 0.0111 while test error is 0.0167

* LDA (normal a priori): Train (apparent) error is 0.0111 while test error is 0.0167

* LDA (uniforme a priori): Train (apparent) error is 0.0222 while test error is 0.0167


### Conclusiones

Utilizando un random seed de 6543, el error en training es más alto en el caso de QDA con distribución uniforme a priori, esto es, el modelo está menos fiteado a los datos de training, en comparación con el modelo de QDA con distribución normal a priori. De todos modos, el test error es similar en ambos casos, por lo que ambos generalizan de manera similar. 

Respecto de LDA con distribución normal a priori vemos que tiene el mismo rendimiento que QDA con distribución normal para el mismo conjunto de datos, tanto en training como en test. Por lo que no podríamos decir que hay un modelo notoriamente mejor que el otro. 

En el caso de LDA con distribución uniforme, vemos que ocurre lo mismo, los errores de training y test son iguales que en QDA con distribución uniforme. 

# Consigna

## Implementación
1. Entrenar un modelo QDA utilizando ahora una *a priori* uniforme ¿Se observan diferencias?¿Por qué?
2. Implementar el modelo LDA, entrenarlo y testearlo contra los mismos sets que QDA ¿Se observan diferencias? ¿Podría decirse que alguno de los dos es notoriamente mejor que el otro?
3. Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Qué se observa?
1. *(Opcional)* En `BaseBayesianClassifier._predict_one` se estima la posteriori de cada clase por separado, a pesar de que la cuenta es siempre la misma (cambiando valores de parámetros, pero no dimensiones). Aprovechando el *broadcasting* de NumPy, se puede reemplazar ese list comprehension por un cálculo *tensorizado* donde tanto medias como varianzas (o inversas) estén "stackeadas" sobre un tercer eje, permitiendo el cálculo en paralelo de todas las clases con un:
`log_posteriori = self.tensor_log_a_priori + self._predict_log_conditionals(x)`. Implementar dicha modificación y mostrar su uso. *Ayuda: los métodos `np.stack` y la documentación del operador [`@`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) son de gran utilidad.*

## Preguntas técnicas

Responder las 7 preguntas que se encuentran distribuidas a lo largo del código.

## Preguntas teóricas

1. En LDA se menciona que la función a maximizar puede ser, mediante operaciones, convertida en:



$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

<span style="color: blue">

Mostrar los pasos por los cuales se llega a dicha expresión. Partimos de la funcion a maximizar de QDA

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

$$
\log{f_j(x)} = - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C'
$$

$$
\log{f_j(x)} =  - \frac{1}{2} [x^T \Sigma^{-1} (x- \mu_j) - \mu_j^T \Sigma^{-1} (x- \mu_j)] + C'
$$

$$
\log{f_j(x)} =  - \frac{1}{2} [x^T \Sigma^{-1} x - x^T \Sigma^{-1} \mu_j - \mu_j^T \Sigma^{-1} x + \mu_j^T \Sigma^{-1} \mu_j] + C'
$$



Como $\Sigma$ es una matriz cuadrada y simétrica, $\Sigma^{-1}$ tambien simétrica y por lo tanto vale que: $x^T \Sigma^{-1} \mu_j = \mu_j^T \Sigma^{-1} x$


$$
\log{f_j(x)} =  - \frac{1}{2} [x^T \Sigma^{-1} x - 2 \mu_j^T \Sigma^{-1} x - \mu_j^T \Sigma^{-1} \mu_j] + C'
$$

$$
\log{f_j(x)} =  - \frac{1}{2} [x^T \Sigma^{-1} x - 2 \mu_j^T \Sigma^{-1} (x - \frac{1}{2} \mu_j)] + C'
$$

$$
\log{f_j(x)} =  - \frac{1}{2} x^T \Sigma^{-1} x + \mu_j^T \Sigma^{-1} (x - \frac{1}{2} \mu_j) + C'
$$     



Ahora, si comparamos dos clases, j y k haciendo el logaritmo del cociente entre ellas, se puede ver que:

$$
\log {\frac {P(G = j|X=x)}{P(G = k|X=x)}} = \log {\frac {f_j(x)}{f_k(x)}}+ \log {\frac {\pi_j}{\pi_k}} 
$$

$$
\log {\frac {P(G = j|X=x)}{P(G = k|X=x)}} = \log {\frac {\pi_j}{\pi_k}} - \frac{1}{2} (\mu_j+\mu_k)^T \Sigma^{-1} (\mu_j-\mu_k)+ x^T\Sigma^{-1} (\mu_j-\mu_k)
$$

La igualdad en la matriz de covarianza entre clases ($\Sigma_j=\Sigma$) hace que el factor de normalización se cancele asi como también la parte cuadratica del exponente.

Esta exprecion linear de probabilidades implica que el limite de decicion entre las clases $j$ y $k$ es lineal en $x$; en $p$ dimenciones un hyperplano. Esto por supuesto es verdadero para cualqueir par de clases por lo tanto todos los limites de decision son lineales. (The Elements of Statistical Learning)

Por lo que al sacar la parte cuadratica, la ecuacion a maximizar de LDA queda de la forma:



$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$

</span>

2. Explicar, utilizando las respectivas funciones a maximizar, por qué QDA y LDA son "quadratic" y "linear".

<span style="color: blue">

Como se pudo observar en las ecuaciones del punto anterior de QDA y LDA:

##### QDA:

$$
\log{f_j(x)} =  -\frac{1}{2}\log |\Sigma| - \frac{1}{2} (x-\mu_j)^T \Sigma^{-1} (x- \mu_j) + C
$$

$$
\log{f_j(x)} =  - \frac{1}{2} x^T \Sigma^{-1} x + \mu_j^T \Sigma^{-1} (x - \frac{1}{2} \mu_j) + C'
$$ 

##### LDA:

$$
\log{f_j(x)} =  \mu_j^T \Sigma^{-1} (x- \frac{1}{2} \mu_j) + C'
$$


La función a maximizar de QDA tiene la parte cuadratica $x^T \Sigma^{-1} x$. Esto quiere decir que los limites de decicion entre las clases se aproximan a una ecuacion cuadratica 



</span>


3. La implementación de QDA estima la probabilidad condicional utilizando `0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x` que no es *exactamente* lo descrito en el apartado teórico ¿Cuáles son las diferencias y por qué son expresiones equivalentes?

`0.5*np.log(det(inv_cov)) -0.5 * unbiased_x.T @ inv_cov @ unbiased_x`

<span style="color: blue">

$$
\log{f_j(x)} = -\frac{1}{2}\log |\Sigma_j| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$

$$
\log{f_j(x)} = \frac{1}{2}\log |\Sigma_j^{-1}| - \frac{1}{2} (x-\mu_j)^T \Sigma_j^{-1} (x- \mu_j) + C
$$


$$
|\Sigma_j| = -|\Sigma_j^{-1}|
$$

</span>

El espíritu de esta componente práctica es la de establecer un mínimo de trabajo aceptable para su entrega; se invita al alumno a explorar otros aspectos que generen curiosidad, sin sentirse de ninguna manera limitado por la consigna.

## Ejercicio teórico

Sea una red neuronal de dos capas, la primera de 3 neuronas y la segunda de 1 con los parámetros inicializados con los siguientes valores:
$$
w^{(1)} = 
\begin{pmatrix}
0.1 & -0.5 \\
-0.3 & -0.9 \\ 
0.8 & 0.02
\end{pmatrix},
b^{(1)} = \begin{pmatrix}
0.1 \\
0.5 \\
0.8 
\end{pmatrix},
w^{(2)} = 
\begin{pmatrix}
-0.4 & 0.2 & -0.5
\end{pmatrix},
b^{(2)} = 0.7
$$

y donde cada capa calcula su salida vía

$$
y^{(i)} = \sigma (w^{(i)} \cdot x^{(i)}+b^{(i)})
$$

donde $\sigma (z) = \frac{1}{1+e^{-z}}$ es la función sigmoidea .

\\
Dada la observación $x=\begin{pmatrix}
1.8 \\
-3.4
\end{pmatrix}$, $y=5$ y la función de costo $J(\theta)=\frac{1}{2}(\hat{y}_\theta-y)^2$, calcular las derivadas de J respecto de cada parámetro $w^{(1)}$, $w^{(2)}$, $b^{(1)}$, $b^{(2)}$.

In [ ]:
º# your code should start here